In [2]:
import mediapipe as mp
import cv2
import numpy as np

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

Saving the Video

In [11]:
import time
cap = cv2.VideoCapture(0)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc('P','I','M','1')
videoWriter = cv2.VideoWriter('distance.avi',fourcc,fps,(int(width),int(height)))

while cap.isOpened():
    ret,frame = cap.read()

    try:
        cv2.imshow('Press',frame)
        videoWriter.write(frame)
    except Exception as e:
        print(e)
        break
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
videoWriter.release()
cv2.destroyAllWindows()

Capturing Landmarks and Saving it to CSV

In [3]:
import csv
import os
import numpy as np
import matplotlib.pyplot as plt

In [66]:
landmarks = ['class']  # Class is Y - target value
for val in range(0,33):
    landmarks+=['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [67]:
with open('coordinates/distance.csv',mode='w',newline='') as f:
    csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [68]:
# Function for exporting the landmarks to train
def export_landmark(results,action):
    try:
        keypoints = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()
        keypoints = np.insert(keypoints,0,action)

        with open('coordinates/distance.csv',mode='a',newline='') as f: # a stands for append
            csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print(e)
        pass

In [14]:
cap = cv2.VideoCapture('videos/distance.avi')
# initiate the holistic model
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,image = cap.read()
        # Recolor the feed
        if image is None: break
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = pose.process(image)

        # Recoloring the image back for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS
                                  , mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))
        
        k = cv2.waitKey(1)
        if k==97: # a key 
            export_landmark(results,-1) # -1 -> narrow 
        if k==115:# s key
            export_landmark(results,0) # 0 -> correct
        if k==100:# d key
            export_landmark(results,1) # 1 -> wide

        cv2.imshow('Raw Cam Feed',image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Training the labelled Data on a Classification Model

In [69]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [70]:
df = pd.read_csv('coordinates/distance.csv')
df.head()

,class,x0,y0,z0,v0,x1,y1,z1,v1,x2,...,z30,v30,x31,y31,z31,v31,x32,y32,z32,v32
0,-1,0.584025,0.159793,-0.292717,0.999987,0.594313,0.141254,-0.266716,0.999960,0.600919,...,0.121014,0.304274,0.627603,1.135382,-0.009953,0.539158,0.574862,1.139200,-0.073454,0.502346
1,-1,0.584042,0.159849,-0.276854,0.999985,0.594403,0.141271,-0.250710,0.999958,0.601043,...,0.121563,0.301967,0.628243,1.138702,-0.008002,0.532657,0.572806,1.140932,-0.073748,0.494412
2,-1,0.583171,0.160286,-0.206906,0.999985,0.594192,0.141636,-0.182152,0.999959,0.600829,...,0.149851,0.302420,0.627015,1.153367,-0.019950,0.523825,0.569784,1.148249,-0.035656,0.487499
3,-1,0.582440,0.160974,-0.242285,0.999925,0.594154,0.142744,-0.218063,0.999876,0.600791,...,0.145155,0.307992,0.625522,1.151539,0.001270,0.521877,0.570017,1.148660,-0.045839,0.488957
4,-1,0.581426,0.161008,-0.258505,0.999880,0.593836,0.142962,-0.234976,0.999812,0.600405,...,0.150651,0.315177,0.625584,1.149821,0.015583,0.523163,0.570213,1.148324,-0.043510,0.492800


Oversampling

In [80]:
import pandas as pd
from sklearn.utils import resample

# Assuming 'df' is your initial DataFrame
# Separate the classes into different DataFrames
df_wide = df[df['class'] == 1]
df_correct = df[df['class'] == 0]
df_narrow = df[df['class'] == -1]

# Upsample the minority classes 'wide' and 'narrow'
df_wide_upsampled = resample(df_wide, 
                             replace=True,     # sample with replacement
                             n_samples=len(df_correct),    # to match majority class
                             random_state=123) # reproducible results

df_narrow_upsampled = resample(df_narrow, 
                               replace=True,     # sample with replacement
                               n_samples=len(df_correct),    # to match majority class
                               random_state=123) # reproducible results

# Combine majority class with upsampled minority classes
df_upsampled = pd.concat([df_correct, df_wide_upsampled, df_narrow_upsampled])

# Display new class counts
print(df_upsampled['class'].value_counts())


class
 0    1267
 1    1267
-1    1267
Name: count, dtype: int64


In [82]:
col = df_upsampled.columns.get_loc('x23')
col

93

In [100]:
df_upsampled.head()

,class,x0,y0,z0,v0,x1,y1,z1,v1,x2,...,z30,v30,x31,y31,z31,v31,x32,y32,z32,v32
1078,0,0.535491,0.157021,-0.496034,0.999997,0.544806,0.134413,-0.476163,0.999990,0.550437,...,0.271446,0.421259,0.670208,1.185598,0.034320,0.674799,0.433560,1.178829,0.049441,0.644085
1079,0,0.535821,0.156723,-0.500807,0.999997,0.545350,0.134231,-0.481266,0.999991,0.551043,...,0.269930,0.424649,0.670092,1.186381,0.010791,0.680771,0.434055,1.180121,0.046715,0.647606
1080,0,0.535024,0.156694,-0.494726,0.999997,0.544695,0.134226,-0.475988,0.999991,0.550452,...,0.267841,0.428512,0.670094,1.187457,-0.000541,0.685497,0.434000,1.180862,0.046221,0.651207
1081,0,0.535314,0.156908,-0.457787,0.999997,0.544918,0.134494,-0.439503,0.999991,0.550692,...,0.240669,0.432296,0.670112,1.188862,-0.013231,0.691477,0.429585,1.183204,0.027164,0.655535
1082,0,0.535629,0.156927,-0.455454,0.999997,0.545170,0.134513,-0.436273,0.999991,0.550874,...,0.226145,0.431457,0.668402,1.188878,-0.023262,0.690622,0.429614,1.183100,0.013385,0.655010


In [83]:
X = df_upsampled.iloc[:,col:].values # features bas 23-32
Y = df_upsampled['class'] # label

In [34]:
X

array([[ 0.48402268,  0.51516253, -0.30524597, ...,  1.13919961,
        -0.0734538 ,  0.5023464 ],
       [ 0.483634  ,  0.45770404, -0.46332392, ...,  1.14093196,
        -0.0737481 ,  0.49441168],
       [ 0.49187917,  0.34390962, -0.44369259, ...,  1.14824879,
        -0.03565636,  0.48749927],
       ...,
       [ 0.39220408,  0.58136964, -0.21910636, ...,  1.18740273,
         0.10085978,  0.59440064],
       [ 0.39107221,  0.56839615, -0.21919695, ...,  1.17085266,
         0.17004454,  0.5587936 ],
       [ 0.39090902,  0.56489682, -0.2243218 , ...,  1.16311038,
         0.14118013,  0.52646887]])

In [84]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=1234)

In [85]:
# Training and Pipelining the Model
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

In [86]:
# Pipelines
pipelines = {
    'rf':make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(),GradientBoostingClassifier())
}

In [87]:
fit_models = {}
for algo,pipeline in pipelines.items():
    model = pipeline.fit(X_train,Y_train)
    fit_models[algo] = model 

In [88]:
fit_models

{'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [89]:
fit_models['rf'].predict(X_test)

array([-1,  1,  0, ...,  1,  0,  1], dtype=int64)

Evaluate and Serialize the Model


In [5]:
from sklearn.metrics import accuracy_score,precision_score,recall_score
import pickle

In [6]:
for algo,model in fit_models.items():
    Y_pred = model.predict(X_test)
    print(algo,accuracy_score(Y_test.values,Y_pred),
          precision_score(Y_test.values,Y_pred,average='weighted'),
          recall_score(Y_test.values,Y_pred,average='weighted'))

NameError: name 'fit_models' is not defined

In [92]:
Y_pred = fit_models['gb'].predict(X_test)
Y_pred

array([-1,  1,  0, ...,  1,  0,  1], dtype=int64)

In [93]:
with open('pickle files/distance.pkl','wb') as f:
    pickle.dump(fit_models['gb'],f)

Making Detections with the Model

In [7]:
with open('pickle files/distance.pkl','rb') as f:
    model = pickle.load(f)

In [8]:
selected_landmarks = [] #  note in all the databases the landmarks are starting from 1 instead of 0
for i in range(23, 33):
    selected_landmarks += [f'x{i}', f'y{i}', f'z{i}', f'v{i}']

In [18]:
import pandas as pd
cap = cv2.VideoCapture(0)
current_Stage = ''
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()
        frame = cv2.resize(frame, (640, 480))
        # Recolor Feed
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = pose.process(image)

        # Recolor it back
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        # Draw Landmarks
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS
                                  , mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))
        
        try:
            row = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            row = row[(23*4):]
            X = pd.DataFrame([row],columns=selected_landmarks)
            bodylang_prob = model.predict_proba(X)[0]
            bodylang_class = model.predict(X)[0]
            
           # Dictionary mapping classes to their respective stages and probability thresholds
            class_thresholds = {
                -1.0: ("Narrow", 0.7),
                0.0: ("Correct", 0.5),
                1.0: ("Wide", 0.7)
            }

            # Get the current stage based on the class and probability
            bodylang_class = model.predict(X)[0]
            max_prob = bodylang_prob[bodylang_prob.argmax()]

            if bodylang_class in class_thresholds:
                stage, threshold = class_thresholds[bodylang_class]
                current_Stage = stage if max_prob > threshold else "Correct"
            else:
                current_Stage = "Correct"

            # Get status box
            cv2.rectangle(image,(0,0),(250,60),(245,117,16),-1)

            # Display Class
            cv2.putText(image,'Class',
                        (95,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,current_Stage,
                            (90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            # Display Probability
            cv2.putText(image,'Prob',
                        (15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str(round(bodylang_prob[np.argmax(bodylang_prob)],2)),
                        (10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            cv2.imshow('Deadlifts Distance',image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        except Exception as e:
            print(e)
            pass   
cap.release()
cv2.destroyAllWindows()

'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'


c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'


c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  

'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'


c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  

'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute

c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
